# Permutation Flow-Shop Scheduling Problem

This is a variant of the Flot-shop scheduling problem (FSSP) in which the sequence of jobs is the same in every machine.

$$
 \begin{align}
     \text{min} \quad & C_{\text{max}} \\
     \text{s.t.} \quad & x_{m-1, j} + p_{m, j} \leq x_{m, j}
         & \forall ~ j \in J; m \in (2, ..., |M|)\\
     & x_{m, j} + p_{m, j} \leq x_{m, k} \lor x_{m, k} + p_{m, k} \leq x_{m, j}
         & \forall ~ j \in J; k \in J, j \neq k\\
     & x_{|M|, j} + p_{|M|, j} \leq C_{\text{max}}
         & \forall ~ j \in J\\
     & x_{m, j} \geq 0 & \forall ~ j \in J; m \in M\\
     & z_{j, k} \in \{0, 1\} & \forall ~ j \in J; k \in J, j \neq k\\
 \end{align}
 $$

 You can compare this implementation to MILP solvers at the [end of the notebook](#bonus---milp-model).

In [1]:
import json
import time

import pandas as pd

from bnbprob.pfssp import CallbackBnB, PermFlowShop
from bnbpy import configure_logfile

In [2]:
configure_logfile("pfssp-experiments.log", mode="w")

In [3]:

solutions = {}
problems = [f'car{i}' for i in range(1, 9)] + [
    f'reC{str(i).zfill(2)}' for i in range(1, 12, 2)
] + [
    f'ta{str(i).zfill(3)}' for i in range(1, 21, 1)
]

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShop.from_p(p, constructive='neh')
        bnb = CallbackBnB(eval_node='in', rtol=0.0001)

        start = time.time()
        sol = bnb.solve(problem, maxiter=5_000_000, timelimit=3600)
        duration = time.time() - start
        solutions[prob] = (sol, duration)

        del bnb
        del problem
        time.sleep(0.1)

In [4]:
# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []

for key, val in solutions.items():
    sol, t = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    time_.append(t)


df = pd.DataFrame({
    "problem": problems_,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "time": time_
})

df.to_excel("bench-pfssp.xlsx")